# Generate data to ML models
Generate the data to ML models. For this example only ficticious data are generated acording a certain formula (also ficticous that try to represent the problem of process that generate an output and this output pass across a tank and then go to another tank)

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Gurobi-ML-tips-modeling


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [3]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import gcsfs
import pickle

In [4]:
# ### desarrollo

# PROJECT_ID = PROJECT_GCP
# ! gcloud config set project $PROJECT_ID

### 0. Paramaters

In [9]:
len_data = 1000

### 1. Read Master Table

In [39]:
path_master_table = f'config/config_ml_models_development/MasterTable.xlsx'
master_table = pd.read_excel(path_master_table)
master_table

,TAG,TAG_DESCRIPTION,DESCRIPCION,CLASIFICACION_NAME,CLASIFICACION,USE_ACTUAL_MODEL,MEAN_VALUE
0,X1,X1,Variable de entrada al proceso A. Variable Pri...,Primary,P,MLA,10
1,O1,O1,Variable de entrada al proceso A. No es una va...,Observed,O,MLA,10
2,O2,O2,Variable de entrada al proceso A. No es una va...,Observed,O,MLA,10
3,O3,O3,Variable de entrada al proceso A. No es una va...,Observed,O,MLA,10
4,Y1,Y1,Variable target del proceso A y Variable de en...,Target,T,MLA,30
5,Z1,Z1,Variable de salida del tanque X y Variable de ...,Secundary,S,MLB,30
6,X2,X2,Variable de entrada al proceso B. Aparece por ...,Primary,P,MLB,10
7,O4,O4,Variable de entrada al proceso B. No es una va...,Observed,O,MLB,10
8,O5,O5,Variable de entrada al proceso B. No es una va...,Observed,O,MLB,10
9,Y2,Y2,Variable target del proceso B (y variable de e...,Target,T,MLB/MLC,30


### 2. Define data random features and target
Define data random and create a formula to map the features into a target and then generate random data to append to the target

#### 2.1 Process A

In [141]:
list_features_process_a = [
    "X1",
    "O1",
    "O2",
    "03"
]

list_target_process_a = ["Y1"]

In [142]:
##### FEATURES

# generate data features process_a
np.random.seed(42)
values_features_process_a = 10 * np.random.rand(len_data, len(list_features_process_a))

In [143]:
#### TARGET

# define formula to map features into a target. np.exp(-1/data) + np.log(data)
formula_target_process_a = (np.exp(-1/values_features_process_a) + np.log(values_features_process_a)).sum(axis = 1)

# generate noise to add to the formula of the target generated
np.random.seed(52)
noise_target_process_a = 10 * np.random.rand(len_data, len(list_target_process_a))

# generate data target process_a
values_target_process_a  = formula_target_process_a.reshape(formula_target_process_a.shape[0], 1) + noise_target_process_a

In [144]:
# generate dataframe process_a
data_process_a = pd.DataFrame(values_features_process_a, columns = list_features_process_a)
data_process_a[list_target_process_a] = values_target_process_a
data_process_a

,X1,O1,O2,03,Y1
0,3.745401,9.507143,7.319939,5.986585,18.968108
1,1.560186,1.559945,0.580836,8.661761,4.889535
2,6.011150,7.080726,0.205845,9.699099,9.174935
3,8.324426,2.123391,1.818250,1.834045,12.928706
4,3.042422,5.247564,4.319450,2.912291,9.334401
...,...,...,...,...,...
995,8.323635,8.056488,2.243186,2.260791,10.888191
996,8.173053,9.303984,0.953609,4.500586,12.010378
997,3.374543,8.706885,0.835249,2.111259,11.360801
998,7.524683,0.512938,4.925354,4.421070,15.351852


#### 2.2 Process B

In [145]:
list_features_process_b = [
    "Z1",
    "X2",
    "O4",
    "O5"
]

list_target_process_b = ["Y2"]

In [155]:
##### FEATURES

# generate data features process_b
np.random.seed(62)
values_features_process_b = 10 * np.random.rand(len_data, len(list_features_process_b))

In [162]:
#### TARGET

# define formula to map features into a target. np.exp(-3/data) + np.log(data)
formula_target_process_b = (np.exp(-3/values_features_process_b) + np.log(values_features_process_b)).sum(axis = 1)

# generate noise to add to the formula of the target generated
np.random.seed(12)
noise_target_process_b = 10 * np.random.rand(len_data, len(list_target_process_b))

# generate data target process_b
values_target_process_b  = formula_target_process_b.reshape(formula_target_process_b.shape[0], 1) + noise_target_process_b

In [163]:
# generate dataframe process_b
data_process_b = pd.DataFrame(values_features_process_b, columns = list_features_process_b)
data_process_b[list_target_process_b] = values_target_process_b
data_process_b

,Z1,X2,O4,O5,Y2
0,0.337547,4.891075,8.460851,4.114020,7.318266
1,6.314154,2.909432,5.428003,2.281814,14.650147
2,2.402763,9.534575,6.458104,0.178572,7.552658
3,9.961258,7.351945,7.219425,4.104965,15.566424
4,2.037802,9.621025,2.244710,0.477828,4.417832
...,...,...,...,...,...
995,2.658250,5.245613,9.591574,2.058272,14.119570
996,6.672634,8.605599,8.763929,6.766465,10.932722
997,3.847030,9.344537,0.500198,9.514672,15.740791
998,3.092017,9.321674,4.503694,0.325089,12.986620


#### 2.3 Process C
This process has its own formula as ml models. To use this formula in gurobi it is necesary define the formula directly as constraint. So, the formula used here is the original formula that will be used as "model"

In [164]:
list_features_process_c = [
    "X3",
    "06",
]

list_target_process_c = ["Y2"]

In [150]:
##### FEATURES

# generate data features process_c
np.random.seed(62)
values_features_process_c = 10 * np.random.rand(len_data, len(list_features_process_c))

In [174]:
#### TARGET

# define formula to map features into a target. np.exp(-1/data) + np.log(data)
formula_target_process_c = 24*1.125*values_features_process_c[:, 0] + values_features_process_c[:, 1]

24*1.125*values_features_process_c[0] + values_features_process_c[1]

# generate noise to add to the formula of the target generated
np.random.seed(12)
noise_target_process_c = 10 * np.random.rand(len_data, len(list_target_process_c))

# generate data target process_c
values_target_process_c  = formula_target_process_c.reshape(formula_target_process_c.shape[0], 1) + noise_target_process_c

In [175]:
values_target_process_c

array([[ 36.39742503],
       [140.46791871],
       [148.89202655],
       [238.65859727],
       [ 37.07123825],
       [ 30.83038687],
       [ 17.65778418],
       [230.37159135],
       [280.65206966],
       [105.16905312],
       [116.43292707],
       [ 19.66407183],
       [131.02330813],
       [222.91297499],
       [251.68091628],
       [209.371372  ],
       [100.41338641],
       [104.69684242],
       [258.36204873],
       [ 75.81818723],
       [221.6834079 ],
       [256.16849136],
       [ 23.95065696],
       [235.70194153],
       [177.33583595],
       [181.2270112 ],
       [112.26161801],
       [ 64.85303366],
       [193.00934355],
       [ 69.79186204],
       [236.81560135],
       [105.87256954],
       [ 57.32345761],
       [ 31.24171717],
       [260.4527519 ],
       [ 22.30724697],
       [212.30651061],
       [122.94074934],
       [224.54637165],
       [232.49605477],
       [ 85.74440817],
       [ 28.68018117],
       [151.75636287],
       [ 77

In [165]:
# generate dataframe process_c
data_process_c = pd.DataFrame(values_features_process_c, columns = list_features_process_c)
data_process_c[list_target_process_c] = values_target_process_c
data_process_c

,X3,06,Y2
0,0.931083,9.716559,4.170220
1,4.838600,2.425227,7.203245
2,5.311238,2.855442,0.001144
3,8.626304,0.411002,3.023326
4,1.083477,7.671600,1.467559
...,...,...,...
995,5.274138,6.326354,9.744035
996,3.121975,5.040939,3.117029
997,4.437344,6.856064,6.687966
998,7.370125,4.491671,3.259672


In [ ]:
df['763F0505_calc'] = (*df['763N0903.PV']*(df['763F0155.SP'] + df['763F0173.SP'])) / (0.9*100)

### 3. Generate Data

### 4. Save Data